In [ ]:
#OBSERVATION !!!! The content of part of this public code was removed in order to preserve
                    # informations about the Anatel database structure and to preserve sensitive data

import pandas as pd
import sqlalchemy
import pyodbc
import nltk 
import sklearn
!pip install stanza
import stanza
import re
!pip install -U gensim
import gensim
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec
import string
import time
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from scipy.stats import randint as sp_randint
!pip install spacy
import spacy
from sklearn.model_selection import cross_val_score
from nltk.stem import RSLPStemmer
!pip install pyodbc 
nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('floresta')
stanza.download('pt')
nltk.download('stopwords')
from nltk.corpus import stopwords 

In [ ]:
def Tokenize(sentence):
    sentence = sentence.lower()
    sentence = nltk.word_tokenize(sentence)
    return sentence

def Stemming(sentence):
    stemmer = RSLPStemmer()
    phrase = []
    for word in sentence:
        phrase.append(stemmer.stem(word.lower()))
    return phrase

#construction a lemmantization using stanza toolkit
nlp = stanza.Pipeline('pt')

def Lemmatize(sentence):
    sentence = sentence.lower()
    lemma = ""
    for sent in nlp(sentence).sentences:
        for word in sent.words:
            lemma += word.lemma + "\t"

    return lemma

#Function that lemmatize the text, tokenize it and than return the list of  tokenized Lemmas 
def LemmatizeTokenized(sentence):
    sentence = sentence.lower()
    lemmas = [word.lemma for sent in nlp(sentence).sentences for word in sent.words]
    return lemmas

stops = set(nltk.corpus.stopwords.words('portuguese') + 
            ['protocolo','reclamada','prestadora','operadora','operador','prestador','empresa',
            'oi','vivo','tim','claro','embratel', 'sky','algar','telefonica','telefônica','net','nextel',
            'dia','dias','data','ano', 'anos','após','fez','referente','consumidor','consumidora','para',
            'pois','porém','anatel','reclama','reclamação','reclamar','providência','pede','telefônico',
            'providências','janeiro','fevereiro','março','abril','maio','junho','julho','agosto',
            'setembro','outubro','novembro','dezembro','providencia','providencias','telefone','contrato','contratar',
            'ser','fiz','ter','sendo','diz','disse','nada','nome','vir','www',
            'numero','número','nr','nº','vc','vcs','pq','tb','ai','aí', '(a)','(es)','!', '?','mês',
             '(',')','*','-','2019','2020', '2021','\"',"\'", '@', '&','#'])

#obs.: LEMBRAR de montar expressão regular para remover: numeros de protocolo, números de telefone e datas
def RemoveStopWords(sentence):
    sentence = Tokenize(sentence)
    phrase = []
    for word in sentence:
        if word not in stops:
            phrase.append(word)
    return phrase

def RemoveStopWordsAndLemmatize(sentence):
    #sentence = sentence.lower() we expect receive a sentence already in lower case
    lemmas = ""
    for sent in nlp(sentence).sentences:
        for word in sent.words:
            if(word.lemma not in stops): lemmas += word.lemma + " "

    return lemmas

def LemmatizeSentence(sentence):
    #sentence = sentence.lower() we expect receive a sentence already in lower case
    lemmas = ""
    for sent in nlp(sentence).sentences:
        for word in sent.words:
            lemmas += word.lemma + " "

    return lemmas

#fraseTeste = Tokenize("Bom dia! Vamos testar este tokenizador Jóia!")
fraseTeste = Tokenize("Vasco da Gama é o maior português de sempre!")

print(fraseTeste)


stemmer = nltk.stem.RSLPStemmer()
print(Stemming(fraseTeste))

sentenceWordsPointRefund = ("ressarcimento ressarcida "+
                            "ressarcido ressarcir ressarça " +
                            "restituição restituicao restitua restituir reembolso reembolse reembolsar reembolsado " +
                            "indenização indenize indenizacao indenizado ")

#describe the words that point a refund by the company
sentenceWordsPointRefundCompany = ("lançamento lancamento desconto reajuste reajustamos devolucao devolvemos devolução " +
                            "devolvido devolvida ressarcimento ressarcida ajuste ajustado ajustada liberado "+
                            "bônus bonus " +
                            "crédito credito créditos creditos creditar " +
                            "isentado isentar")

#words that can point to a refund. Let's introduce a lemmatization process
setWordsRefound = Tokenize(sentenceWordsPointRefund)
print(setWordsRefound)

#stemming words point to a refund
stemmedWordsRefund = Stemming(setWordsRefound)
print(stemmedWordsRefund)

#lemmatize words to a refund
lemmatizedWordsRefund = Tokenize(Lemmatize(sentenceWordsPointRefund))


setLemmatizedWordsRefund = set(['ressarcimento','ressarcidar', 'ressarcir', 'ressarçar', 'restituição',
                             'restituicao', 'restituir',  'reembolso', 'reembolse', 'reembolsar',
                             'indenização', 'indenizar', 'indenizacao', 'estorno', 'estornar', 'extorno',
                              'extornar', 'extorne', 'estorne','devolução','devolucao','devolver', 'crédito','credito',
                              'creditar', 'compensação','compensar','reparação', 'reparar','reaver', 'multar','fidelidade',
                              'valor','absurdo', 'desconto', 'redução','roubo', 'preço','negociar','debito', 'juro',
                              'caro','contar','indevidamente', 'indevido','isenção'])

setLemmatizedWordsRefundCompany = set(['lançar','abatimento','isento','isenção','conceder','baixo','acordo','desconto',
                                       'credito','renovação','ajuste','contestar','lançar','ressarcir','corrigir','diferença',
                                       'readequação','desconsiderar','concessão', 'resolver','baixo','cancelamento',
                                       'providenciar'])

setLemmasPointSum = set(['de','e'])

setLemmasPointSubtraction = set(['para'])


#words that can point to a refund in the answers. Let's introduce a lemmatization process
setWordsRefundCompany = Tokenize(sentenceWordsPointRefundCompany)
print(setWordsRefundCompany)

#stemming words point to a refund in a company answer
stemmedWordsRefundCompany = Stemming(setWordsRefundCompany)
print(setWordsRefundCompany)

#lemmatize words to a refund in a company answer
lemmatizedWordsRefundCompany = Tokenize(Lemmatize(sentenceWordsPointRefundCompany))



#function to check if there is some word that point a refund inside the complaint.
def checkForRefundWords(complaint):
    tokensComplaint = Tokenize(complaint)
    stemmedWordsComplaint = Stemming(tokensComplaint)
    
    for steamInComplaint in stemmedWordsComplaint:
        if steamInComplaint in stemmedWordsRefund:
            print("Stemming detected "+steamInComplaint)
            return True
    return False

def checkForRefundWordsUsingLemma(complaint):
    lemmatizedWordsComplaint = Lemmatize(complaint)
    tokensComplaint = Tokenize(lemmatizedWordsComplaint)
    
    for lemmaInComplaint in tokensComplaint:
        if lemmaInComplaint in lemmatizedWordsRefund:
            print("Lemma detected "+lemmaInComplaint)
            return True
    return False

def checkForRefundWordsUsingLemma2(complaint):
    tokensComplaint = LemmatizeTokenized(complaint)
    
    for lemmaInComplaint in tokensComplaint:
        if lemmaInComplaint in setLemmatizedWordsRefund:
            print("Lemma detected "+lemmaInComplaint)
            return 1
    return 0

#function to check if there is some word that point a refund inside the COMPANY ANSWER.
def checkForRefundWordCompany(answers):
    tokensAnswers = Tokenize(answers)
    stemmedWordsAnswers= Stemming(tokensAnswers)
    
    for steamInComplaint in stemmedWordsAnswers:
        if steamInAnswer in stemmedWordsRefundCompany:
            print("Stemming answer detected "+steamInAnswer)
            return True
    return False

def checkForRefundWordsCompanyUsingLemma(answers):
    lemmatizedWordsAnswers = Lemmatize(answers)
    tokensAnswers = Tokenize(lemmatizedWordsAnswers)
    
    for lemmaInAnswer in tokensAnswers:
        if lemmaInAnswer in lemmatizedWordsRefundCompany:
            print("Lemma answer detected "+lemmaInAnswer)
            return True
    return False

def checkForRefundWordsCompanyUsingLemma2(answers):
    tokensAnswers = LemmatizeTokenized(answers)
    
    for lemmaInAnswer in tokensAnswers:
        if lemmaInAnswer in setLemmatizedWordsRefundCompany:
            print("Lemma answer detected "+lemmaInAnswer)
            return 1
    return 0

def preprocess_text(textComplaint, includeStopWords):
       
    textComplaint = textComplaint.lower()
    
    #first we remove string that represent protocols number, dates and telephone numbers
    #removing protocol numbers
    p1 = re.compile('\d{15}') #removind protocol numbers with 15 digits
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{14}') #removind protocol numbers with 14 digits
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{13}') #removind protocol numbers with 13 digits
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{2}/\d{2}/\d{4}') #removind dates: format dd/mm/yyyy
    textComplaint = p1.sub('', textComplaint)
                        
    p1 = re.compile('\d{2}/\d{2}') #removind short dates: format dd/mm
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{2}h\d{2}') #removind times: format examples 14h35  
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{2}:\d{2}') #removind times: format examples 14:35 
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{1}h\d{2}') #removind times: format examples 9h35  
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{1}:\d{2}') #removind times: format examples 9:35 
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\(\d{2}\)') #removind telephone number area code: format (11)
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{2}\s\d{9}')   # remoing numbers like Example 21 985883880
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{2}\s\d{8}')   # remoing numbers like Example 21 985883880
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{7}-\d{4}')   # remoing numbers like Example 2197033-4392
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{6}-\d{4}')   # removing numbers like Example 213321-4392
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{7}\s\d{4}')   # removing numbers like Example 2197033 4392
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{6}\s\d{4}')   # removing numbers like Example 213321 4392
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{2}\s\d{5}-\d{4}')   # removing numbers like Example 31 98131-3131
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{2}\s\d{4}-\d{4}')   # removing numbers like Example 31 3321-3131
    textComplaint = p1.sub('', textComplaint)
    
    
    p1 = re.compile('\d{2}\s\d{5}\s\d{4}')   # removing numbers like Example 31 98131 3131
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{2}\s\d{4}\s\d{4}')   # removing numbers like Example 31 3321 3131
    textComplaint = p1.sub('', textComplaint)
    
    
    p1 = re.compile('\d{11}|\d{10}|\d{9}|\d{8}') #removind telephone numbers 8 to 11 digits: format 11982165069
    textComplaint = p1.sub('', textComplaint)

      
    p1 = re.compile('\d{5}-\d{4}') #removind telephone numbers like format 99991-0621
    textComplaint = p1.sub('', textComplaint)

    p1 = re.compile('\d{4}-\d{4}') #removind telephone numbers like format 9991-0621
    textComplaint = p1.sub('', textComplaint)
    
    p1 = re.compile('\d{5}\s\d{4}') #removind telephone numbers like format 99991 0621
    textComplaint = p1.sub('', textComplaint)

    p1 = re.compile('\d{4}\s\d{4}') #removind telephone numbers like format 9991 0621
    textComplaint = p1.sub('', textComplaint)   
    
    #than we lemmatize
    #textLemmatized = Lemmatize(textComplaint)
    
    #than we remove stop words over the lemmatized text. Inside the stop words function there is a tokenization and lower case
    lst_text = ""
    if(includeStopWords): lst_text = LemmatizeSentence(textComplaint) #case where we don`t remove stop words
    else: lst_text = RemoveStopWordsAndLemmatize(textComplaint) # case where we remove stop words
    
    #than we return the new string
    ## back to string from list
    #text = " ".join(lst_text)
    #return text
    return lst_text

#function that combine the complaint and the answer, and then 
def getPreprocessFullComplaint(textComplaint, answers):
    fullComplaint = str(textComplaint) + " "+ str(answers)
    return preprocess_text(fullComplaint,False)



In [ ]:
#OBSERVATION !!!! The content of this function was removed from this public code
                    #in order to preserve informations about the Anatel database structure

#lets introduce here the analysis of the company response, in order to introduce in the train and test dataframes
# the column containing the answers to the complaint
#function to obtains the complaint answers given an specific complaint part of the answer
# we will chek also if the part of the answer is not equal to the complaint. If it is, we do not include in answers
def returnComplaintAnswers(idComplaint, textComplaint):
    complaintAnswers = ""
   
    #OBSERVATION !!!! The content of this function was removed from this public code
                    #in order to preserve informations about the Anatel database structure
    
    return complaintAnswers


In [ ]:
# init callback class
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        elif self.epoch % 100 == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        
        
        self.epoch += 1
        self.loss_previous_step = loss


In [ ]:
# init word2vec class
w2v_model = Word2Vec(vector_size = 300, #The number of dimensions of the embedding
                     window = 10, #The maximum distance between a target word and words around the target word.
                     min_count = 2, #The minimum count of words to consider when training the model; words with an occurrence less than this count will be ignored.
                     workers = 16, #The number of threads to use while training
                     sg = 1, #The training algorithm, either CBOW (0) or skip gram (1)
                     negative = 5,
                     sample = 1e-5)
# build vovab
w2v_model.build_vocab(tokenized_corpus)

# train the w2v model
start = time.time()
w2v_model.train(tokenized_corpus, 
                total_examples=w2v_model.corpus_count, 
                epochs=1001, 
                report_delay=1,
                compute_loss = True, # set compute_loss = True
                callbacks=[callback()]) # add the callback class
end = time.time()


print("elapsedtime in seconds to train the embedding model :"+ str(end - start))
# save the word2vec model
w2v_model.save('C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/word2vec.model')

In [ ]:
reloaded_w2v_model = Word2Vec.load('C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/word2vec.model')

words = list(reloaded_w2v_model.wv.key_to_index)
print('Vocab size: '+str(len(words)))
w1 = 'valor'
print("Top 3 words similar to valor:",\
      reloaded_w2v_model.wv.most_similar(positive = w1,topn =3))
w1 = 'restituição'
print("Top 3 words similar to restituição:",\
      reloaded_w2v_model.wv.most_similar(positive = w1,topn =3))
print("Similarity between reembolso and valor:"+\
      str(reloaded_w2v_model.wv.similarity(w1="reembolso",w2="valor")))
print("Similarity between ressarcimento and indignar:"+\
      str(reloaded_w2v_model.wv.similarity(w1="ressarcimento",w2="indignar")))

In [ ]:
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for word in model.wv.key_to_index:
        vectors.append(model.wv[word])
        labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    #labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(reloaded_w2v_model)


In [ ]:
def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    
    
    indices = list(range(len(labels)))
    #selected_indices = random.sample(indices, 25)
    selected_indices=[]
    index = labels.index("reembolso")
    selected_indices.append(index)
    index = labels.index("quantia")
    selected_indices.append(index)
    index = labels.index("valor")
    selected_indices.append(index)
    index = labels.index("dinheiro")
    selected_indices.append(index)
    index = labels.index("abuso")
    selected_indices.append(index)
    index = labels.index("devolução")
    selected_indices.append(index)
    index = labels.index("crédito")
    selected_indices.append(index)
    index = labels.index("ressarcimento")
    selected_indices.append(index)
    index = labels.index("indenização")
    selected_indices.append(index)
    index = labels.index("restituição")
    selected_indices.append(index)
    
    
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))



plot_function = plot_with_matplotlib


plot_function(x_vals, y_vals, labels)

In [ ]:
import numpy as np

#function to vectorize sentences
def sent_vectorizer(sent, model):
    sent_vec =[]
    numw = 0
    for w in sent:
        try:
            if numw == 0:
                sent_vec = model.wv[w]
            else:
                sent_vec = np.add(sent_vec, model.wv[w])
            numw+=1
        except:
            pass

    return np.asarray(sent_vec) / numw

In [ ]:
#function to create the vectors over the pre-process (cleaned) text
STATIC_FLAG_FULL_COMPLAINT = 0
STATIC_FLAG_COMPLAINT = 1
STATIC_FLAG_ANSWERS = 2
model_gensim = reloaded_w2v_model
def createDocsVectors(dataFrame, optionText):

    #testing new approach
    docs_vectors = pd.DataFrame() # creating empty final dataframe
    if(optionText == STATIC_FLAG_FULL_COMPLAINT):textComplaints = dataFrame['TEXT_CLEAN_FULL_COMPLAINT'].values
    if(optionText == STATIC_FLAG_COMPLAINT):textComplaints = dataFrame['TEXT_CLEAN_COMPLAINT'].values
    if(optionText == STATIC_FLAG_ANSWERS):textComplaints = dataFrame['TEXT_CLEAN_ANSWERS'].values

    for text in textComplaints: # looping through each document and cleaning it
        temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
        for word in text.split(' '): # looping through each word of a single document and spliting through space
                try:
                    #word_vec = embeddings[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                    word_vec = sent_vectorizer(word, model_gensim)
                    temp = temp.append(pd.Series(word_vec), ignore_index = True) # if word is present then append it to temporary dataframe
                except:
                    pass
        doc_vector = temp.mean() # take the average of each column(w0, w1, w2,........w300)
        docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe
    return docs_vectors


In [ ]:
#Analizing the heuristic using special words. Here we creaete dataframes to store the results of the heuristic
COLUNAS = [
    'HAS_WORD_POINT_REFUND',
    'HAS_WORD_POINT_REFUND_AGREE'
]

dataFrameHeuristicTrain = pd.DataFrame(columns=COLUNAS)
dataFrameHeuristicTest = pd.DataFrame(columns=COLUNAS)

dataFrameTrain = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/dataFrameTrain1200PreProcessed.csv', sep='\t')
dataFrameTest = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/dataFrameTest.csv', sep='\t')

dataFrameHeuristicTrain['HAS_WORD_POINT_REFUND'] = dataFrameTrain['TEXT_CLEAN_COMPLAINT'].apply(checkForRefundWordsUsingLemma2)
dataFrameHeuristicTrain['HAS_WORD_POINT_REFUND_AGREE'] = dataFrameTrain['TEXT_CLEAN_ANSWERS'].apply(checkForRefundWordsCompanyUsingLemma2)

dataFrameHeuristicTrain.to_csv('C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/dataFrameHeuristicTrain.csv', sep='\t', index=False)

dataFrameHeuristicTest['HAS_WORD_POINT_REFUND'] = dataFrameTest['TEXT_CLEAN_COMPLAINT'].apply(checkForRefundWordsUsingLemma2)
dataFrameHeuristicTest['HAS_WORD_POINT_REFUND_AGREE'] = dataFrameTest['TEXT_CLEAN_ANSWERS'].apply(checkForRefundWordsCompanyUsingLemma2)
dataFrameHeuristicTest.to_csv('C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/dataFrameHeuristicTest.csv', sep='\t', index=False)


In [ ]:
#lets show the accuracy of the heuristic over  COMPLAINTS
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score

#y1 = dataFrame['INCLUDE_REFUND'].values #case train
print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for Sample of '+ str(len(dataFrame)) +' COMPLAINTS using Just special words: ')
accuracy = accuracy_score(y1,dataFrameHeuristic['HAS_WORD_POINT_REFUND'])
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(y1,dataFrameHeuristic['HAS_WORD_POINT_REFUND']))

f1 = sklearn.metrics.f1_score(y1, dataFrameHeuristic['HAS_WORD_POINT_REFUND'])
print('F1 score: %f' % f1)
recall = recall_score(y1, dataFrameHeuristic['HAS_WORD_POINT_REFUND'])
print('Recall score: %f' % recall)
precision = precision_score(y1, dataFrameHeuristic['HAS_WORD_POINT_REFUND'])
print('Precision score: %f' % precision)
print('\n')

#lets show the accuracy of the heuristic over 1200 ANSWERS
#y2 = dataFrame['REFUND_AGREE'].values#case train
print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for Sample of '+ str(len(dataFrame)) +' ANSWERS using Just special words: ')
accuracy = accuracy_score(y2,dataFrameHeuristic['HAS_WORD_POINT_REFUND_AGREE'])
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(y2,dataFrameHeuristic['HAS_WORD_POINT_REFUND_AGREE']))

f1 = sklearn.metrics.f1_score(y2, dataFrameHeuristic['HAS_WORD_POINT_REFUND_AGREE'])
print('F1 score: %f' % f1)
recall = recall_score(y2, dataFrameHeuristic['HAS_WORD_POINT_REFUND_AGREE'])
print('Recall score: %f' % recall)
precision = precision_score(y2, dataFrameHeuristic['HAS_WORD_POINT_REFUND_AGREE'])
print('Precision score: %f' % precision)


In [ ]:
#setting cases where there is a complaint for refund in the ID's below
#functin to tag the complains that includ an asking for refund 
def tagRefund(id):
    return taggedFrames.loc[id]['CLASSIFICACAO']


#functin to tag the complains that includ a custumer asking for refund. Here we consult the by hand tagged dataFrame(taggedFrames)
def tagRefundAsked(id):
    return taggedFrames.loc[id]['REEMBOLSO_SOLICITADO']

#functin to inlcude the refund value complained by the customers, based on the id. Here we consult
# the by hand tagged dataFrame(taggedFrames)
def tagValueComplainded(id):
     return taggedFrames.loc[id]['VALOR_SOLICITADO']

#functin to tag the complains that includ a company's agreement for refund. Here we consult the by hand tagged dataFrame(taggedFrames)
def tagRefundAgree(id):
    return taggedFrames.loc[id]['REEMBOLSO_CONCEDIDO']

#functin to inlcude the value refunded by the companies, based on the id. Here we consult
# the by hand tagged dataFrame(taggedFrames)
def tagValueRefunded(id):
     return taggedFrames.loc[id]['VALOR_CONCEDIDO']


In [ ]:
#dataFrame['INCLUDE_REFUND'] = dataFrame['ID'].apply(tagRefund)    

#here, we include a new column in the dataFrame, containing if the customer asked for a refund
#dataFrame['REFUND_ASKED'] = dataFrame['ID'].apply(tagRefundAsked)    

#here, we include a new column in the dataFrame, containing the value complained for refund
#dataFrame['REAL_VALUE_COMPLAINED'] = dataFrame['ID'].apply(tagValueComplainded)    

#here, we include a new column in the dataFrame, containing the answers to the complaint 
#dataFrame['ANSWERS'] = dataFrame.apply(lambda x: returnComplaintAnswers(x['ID'],x['REGISTRODESCRICAO']), axis=1) 

#here, we include a new column in the dataFrame, containing the information IF the company has offered some refund
dataFrame['REFUND_AGREE'] = dataFrame['ID'].apply(tagRefundAgree)    

#here, we include a new column in the dataFrame, containing the value the company has refunded
#dataFrame['REAL_VALUE_REFUNDED'] = dataFrame['ID'].apply(tagValueRefunded) 

In [ ]:
docs_vectors = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/docsVectors1200Complaints.csv', sep='\t')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

train_x, test_x, train_y, test_y = train_test_split(docs_vectors.drop('INCLUDE_REFUND', axis = 1),
                                                   docs_vectors['INCLUDE_REFUND'],
                                                   test_size = 0.2,
                                                   random_state = 1)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score

model = AdaBoostClassifier(n_estimators=800, random_state = 1)
model.fit(train_x, train_y)
test_pred = model.predict(test_x)

print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for the AdaBoostClassifier in TRAIN sample of '+str(len(dataFrame)) +' complaints: ')
accuracy = accuracy_score(test_y, test_pred)
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(test_y,test_pred))

f1 = sklearn.metrics.f1_score(test_y,test_pred)
print('F1 score: %f' % f1)
recall = recall_score(test_y,test_pred)
print('Recall score: %f' % recall)
precision = precision_score(test_y,test_pred)
print('Precision score: %f' % precision)

In [ ]:
#now we apply the classification model over the test sample of 400 complaints
docs_vectorsTest = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/docsVectors400TestComplaints.csv', sep='\t')
test_x = docs_vectorsTest
test_y = taggedDataFrameTest['REEMBOLSO_SOLICITADO'].values
test_pred = model.predict(test_x)


print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for the AdaBoostClassifier in TEST sample of '+str(len(test_x)) +' complaints: ')
accuracy = accuracy_score(test_y, test_pred)
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(test_y,test_pred))

f1 = sklearn.metrics.f1_score(test_y,test_pred)
print('F1 score: %f' % f1)
recall = recall_score(test_y,test_pred)
print('Recall score: %f' % recall)
precision = precision_score(test_y,test_pred)
print('Precision score: %f' % precision)

In [ ]:

#functions definition to obtain the accuracy (exact match), FP, FN, TN  TP from values predictions 
def checkMach(realValue, predictedValue):
    if realValue == predictedValue:
        return 1
    return 0

def checkFalsePositive(realValue, predictedValue):
    if ( (realValue == 0) and (predictedValue !=0) ):
        return 1
    return 0

def checkFalseNegative(realValue, predictedValue):
    if ( (realValue != 0) and (predictedValue ==0) ):
        return 1
    return 0

def checkTruePositive(realValue, predictedValue):
    if ( (realValue != 0) and (predictedValue !=0) ):
        return 1
    return 0

def checkTrueNegative(realValue, predictedValue):
    if ( (realValue == 0) and (predictedValue ==0) ):
        return 1
    return 0

real_V = 8.3
pred_V = 8.3
match = checkMach(real_V, pred_V)
falsePositive = checkFalsePositive(real_V, pred_V)
falseNegative = checkFalseNegative(real_V, pred_V)
truePositive = checkTruePositive(real_V, pred_V)
trueNegative = checkTrueNegative(real_V, pred_V)

print('Real Value: %f' % real_V)
print('Predicted Value: %f' % pred_V)
print('Mach score: %f' % match)
print('False Positive: %f' % falsePositive)
print('False Negative: %f' % falseNegative)
print('True Positive: %f' % truePositive)
print('True Negative: %f' % trueNegative)


In [ ]:
#now, for the complaints that has been predictec as containing an asking for refund, we will include the values complained
dataFrameTest = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/dataFrameTest.csv', sep='\t')
#predictionsIncludeAskForRefund = test_pred

#dataFrameTest['TEXT_CLEAN_COMPLAINT_WITH_STOP_WORDS'] = dataFrameTest.apply(lambda x: preprocess_text(x['REGISTRODESCRICAO'],True), axis=1)
#dataFrameTest['TEXT_CLEAN_ANSWERS_WITH_STOP_WORDS'] = dataFrameTest.apply(lambda x: preprocess_text(x['ANSWERS'],True), axis=1)


dataFrameTest2 = dataFrameTest 
#including in the test data frame the predictions if is a compaint that includes asking for refund
dataFrameTest2['REAL_VALUE_COMPLAINED'] = taggedDataFrameTest['VALOR_SOLICITADO'] # the true values
dataFrameTest2['ESTIMATED_REFUND_COMPLAINT'] =  taggedDataFrameTest['REEMBOLSO_SOLICITADO'] # include just the true values, to focus this task just in get the value

#in this context, we will use as "predicted values" for cases that involves some Customer ask for complaint the true values
predictionsIncludeAskForRefund = taggedDataFrameTest['REEMBOLSO_SOLICITADO']

#including in the test data frame the predictions for asked refund values
#dataFrameTest2['ESTIMATED_VALUE_COMPLAINED'] = dataFrameTest2.apply(lambda x: getRefundAskedValueInComplaint(x['TEXT_CLEAN_COMPLAINT'], setLemmatizedWordsRefund), axis=1)
dataFrameTest2['ESTIMATED_VALUE_COMPLAINED'] = dataFrameTest2.apply(lambda x: getRefundAskedValueInComplaint(x['TEXT_CLEAN_COMPLAINT_WITH_STOP_WORDS'], setLemmatizedWordsRefund), axis=1)
 
    
realValuesComplained = taggedDataFrameTest['VALOR_SOLICITADO'].values
predictedValuesComplained =dataFrameTest2['ESTIMATED_VALUE_COMPLAINED'].values


dataFrameAccuracyValuesComplained= pd.DataFrame(list(zip(predictionsIncludeAskForRefund,realValuesComplained,predictedValuesComplained)), columns = ['ESTIMATED_REFUND_COMPLAINT','REAL_VALUE_COMPLAINED','ESTIMATED_VALUE_COMPLAINED'])

dataFrameAccuracyValuesComplained2 = dataFrameAccuracyValuesComplained.loc[dataFrameAccuracyValuesComplained['ESTIMATED_REFUND_COMPLAINT']==1]


print('Mean Absolute Error(MAE), Mean Squared Error(MSE) and Median Absolute Deviation(MAD) for Complaints Values predicted over TEST sample of '+str(len(dataFrameAccuracyValuesComplained2)) +' COMPLAINTS: ')
from sklearn.metrics import median_absolute_error, mean_absolute_error, mean_squared_error
y_true = dataFrameAccuracyValuesComplained2['REAL_VALUE_COMPLAINED'].values
y_pred = dataFrameAccuracyValuesComplained2['ESTIMATED_VALUE_COMPLAINED'].values
mae = mean_absolute_error(y_true, y_pred)
print('MAE : %f' %mae)
mse = mean_squared_error(y_true, y_pred)
print('MSE : %f' %mse)
mad = median_absolute_error(y_true, y_pred)
print('MAD : %f' %mad)

dataFrameAccuracyValuesComplained2['ACCURACY'] = dataFrameAccuracyValuesComplained2.apply(lambda x: checkMach(x['REAL_VALUE_COMPLAINED'], x['ESTIMATED_VALUE_COMPLAINED']), axis=1)
dataFrameAccuracyValuesComplained2['FALSE_POSITIVE'] = dataFrameAccuracyValuesComplained2.apply(lambda x: checkFalsePositive(x['REAL_VALUE_COMPLAINED'], x['ESTIMATED_VALUE_COMPLAINED']), axis=1)
dataFrameAccuracyValuesComplained2['FALSE_NEGATIVE'] = dataFrameAccuracyValuesComplained2.apply(lambda x: checkFalseNegative(x['REAL_VALUE_COMPLAINED'], x['ESTIMATED_VALUE_COMPLAINED']), axis=1)
dataFrameAccuracyValuesComplained2['TRUE_POSITIVE'] = dataFrameAccuracyValuesComplained2.apply(lambda x: checkTruePositive(x['REAL_VALUE_COMPLAINED'], x['ESTIMATED_VALUE_COMPLAINED']), axis=1)
dataFrameAccuracyValuesComplained2['TRUE_NEGATIVE'] = dataFrameAccuracyValuesComplained2.apply(lambda x: checkTrueNegative(x['REAL_VALUE_COMPLAINED'], x['ESTIMATED_VALUE_COMPLAINED']), axis=1)

mean = dataFrameAccuracyValuesComplained2['ACCURACY'].mean()
print('ACCURACY Mach : %f' %mean)
falsePositive_sum = dataFrameAccuracyValuesComplained2['FALSE_POSITIVE'].sum()
falseNegative_sum = dataFrameAccuracyValuesComplained2['FALSE_NEGATIVE'].sum()
truePositive_sum = dataFrameAccuracyValuesComplained2['TRUE_POSITIVE'].sum()
trueNegative_sum = dataFrameAccuracyValuesComplained2['TRUE_NEGATIVE'].sum()
recall = truePositive_sum/(truePositive_sum + falseNegative_sum) #recall = tp / (tp + fn)
precision = truePositive_sum/(truePositive_sum + falsePositive_sum)  #precision = tp / (tp + fp)
f1 = 2 * (precision * recall) / (precision + recall) # f1 = 2 * (precision * recall) / (precision + recall)
print('Recall Score : %f' %recall)
print('Precision Score : %f' %precision)
print('F1 Score : %f' %f1)


In [ ]:
#PREDICTION THe VALUES FOR THE COMPANIES ANSWERS
#now, for the ANSWERS that has been predictec as containing an asking for refund, we will include the values complained
dataFrameTest = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/dataFrameTest.csv', sep='\t')
#predictionsCasesRefunded = test_pred

dataFrameTest2 = dataFrameTest 
#including in the test data frame the predictions if is a answer that includes refund
dataFrameTest2['REAL_VALUE_REFUNDED'] = taggedDataFrameTest['VALOR_CONCEDIDO'] # the true values
#dataFrameTest2['ESTIMATED_REFUND_ANSWER'] = predictionsCasesRefunded
dataFrameTest2['ESTIMATED_REFUND_ANSWER'] = taggedDataFrameTest['REEMBOLSO_CONCEDIDO'] # include just the true values, to focus this task just in get the value
#in this context, we will use as "predicted values" for cases that involves some company refund the true values
predictionsCasesRefunded = taggedDataFrameTest['REEMBOLSO_CONCEDIDO']

#including in the test data frame the predictions for refund values
#dataFrameTest2['ESTIMATED_VALUE_REFUNDED'] = dataFrameTest2.apply(lambda x: getRefundAskedValueInComplaint(x['TEXT_CLEAN_ANSWERS'], setLemmatizedWordsRefundCompany), axis=1)
dataFrameTest2['ESTIMATED_VALUE_REFUNDED'] = dataFrameTest2.apply(lambda x: getRefundAskedValueInComplaint(x['TEXT_CLEAN_ANSWERS_WITH_STOP_WORDS'], setLemmatizedWordsRefundCompany), axis=1)



realValuesRefunded = taggedDataFrameTest['VALOR_CONCEDIDO'].values
predictedValuesRefunded =dataFrameTest2['ESTIMATED_VALUE_REFUNDED'].values


#removing lines where there is no preview of asking complaint

dataFrameAccuracyValuesRefunded= pd.DataFrame(list(zip(predictionsCasesRefunded,realValuesRefunded,predictedValuesRefunded)), columns = ['ESTIMATED_REFUND_ANSWER','REAL_VALUE_REFUNDED','ESTIMATED_VALUE_REFUNDED'])

dataFrameAccuracyValuesRefunded2 = dataFrameAccuracyValuesRefunded.loc[dataFrameAccuracyValuesRefunded['ESTIMATED_REFUND_ANSWER']==1]

print('Mean Absolute Error(MAE), Mean Squared Error(MSE) and Median Absolute Deviation(MAD) for REFUNDED VALUES predicted over TEST sample of '+str(len(dataFrameAccuracyValuesRefunded2)) +' ANSWERS: ')
from sklearn.metrics import median_absolute_error, mean_absolute_error, mean_squared_error
y_true = dataFrameAccuracyValuesRefunded2['REAL_VALUE_REFUNDED'].values
y_pred = dataFrameAccuracyValuesRefunded2['ESTIMATED_VALUE_REFUNDED'].values
mae = mean_absolute_error(y_true, y_pred)
print('MAE : %f' %mae)
mse = mean_squared_error(y_true, y_pred)
print('MSE : %f' %mse)
mad = median_absolute_error(y_true, y_pred)
print('MAD : %f' %mad)
 
dataFrameAccuracyValuesRefunded2['ACCURACY'] = dataFrameAccuracyValuesRefunded2.apply(lambda x: checkMach(x['REAL_VALUE_REFUNDED'], x['ESTIMATED_VALUE_REFUNDED']), axis=1)

dataFrameAccuracyValuesRefunded2['FALSE_POSITIVE'] = dataFrameAccuracyValuesRefunded2.apply(lambda x: checkFalsePositive(x['REAL_VALUE_REFUNDED'], x['ESTIMATED_VALUE_REFUNDED']), axis=1)
dataFrameAccuracyValuesRefunded2['FALSE_NEGATIVE'] = dataFrameAccuracyValuesRefunded2.apply(lambda x: checkFalseNegative(x['REAL_VALUE_REFUNDED'], x['ESTIMATED_VALUE_REFUNDED']), axis=1)
dataFrameAccuracyValuesRefunded2['TRUE_POSITIVE'] = dataFrameAccuracyValuesRefunded2.apply(lambda x: checkTruePositive(x['REAL_VALUE_REFUNDED'], x['ESTIMATED_VALUE_REFUNDED']), axis=1)
dataFrameAccuracyValuesRefunded2['TRUE_NEGATIVE'] = dataFrameAccuracyValuesRefunded2.apply(lambda x: checkTrueNegative(x['REAL_VALUE_REFUNDED'], x['ESTIMATED_VALUE_REFUNDED']), axis=1)

mean = dataFrameAccuracyValuesRefunded2['ACCURACY'].mean()
print('ACCURACY Mach : %f' %mean)

falsePositive_sum = dataFrameAccuracyValuesRefunded2['FALSE_POSITIVE'].sum()
falseNegative_sum = dataFrameAccuracyValuesRefunded2['FALSE_NEGATIVE'].sum()
truePositive_sum = dataFrameAccuracyValuesRefunded2['TRUE_POSITIVE'].sum()
trueNegative_sum = dataFrameAccuracyValuesRefunded2['TRUE_NEGATIVE'].sum()
recall = truePositive_sum/(truePositive_sum + falseNegative_sum) #recall = tp / (tp + fn)
precision = truePositive_sum/(truePositive_sum + falsePositive_sum)  #precision = tp / (tp + fp)
f1 = 2 * (precision * recall) / (precision + recall) # f1 = 2 * (precision * recall) / (precision + recall)
print('Recall Score : %f' %recall)
print('Precision Score : %f' %precision)
print('F1 Score : %f' %f1)


In [ ]:
#Part-of-Speech Tagging
#using the MAC-MORPHO Brazilian Portuguese POS-tagged
nltk.corpus.mac_morpho.words()
#['Jersei', 'atinge', 'm\xe9dia', 'de', 'Cr$', '1,4', ...]
nltk.corpus.mac_morpho.sents() 
#[['Jersei', 'atinge', 'm\xe9dia', 'de', 'Cr$', '1,4', 'milh\xe3o',
#'em', 'a', 'venda', 'de', 'a', 'Pinhal', 'em', 'S\xe3o', 'Paulo'],
#['Programe', 'sua', 'viagem', 'a', 'a', 'Exposi\xe7\xe3o', 'Nacional',
#'do', 'Zebu', ',', 'que', 'come\xe7a', 'dia', '25'], ...]
nltk.corpus.mac_morpho.tagged_words()
#[('Jersei', 'N'), ('atinge', 'V'), ('m\xe9dia', 'N'), ...]
nltk.corpus.mac_morpho.tagged_sents()

#lets contruct our 3-gram tagger from MAC_MORPHO Corpus
corpusSentences = nltk.corpus.mac_morpho.tagged_sents()
#Lets divide into train and test datasets
size = int(len(corpusSentences) * 0.9)
size
train_sentences = corpusSentences[:size]
test_sentences = corpusSentences[size:]

tagger0 = nltk.DefaultTagger('N')
tagger1 = nltk.UnigramTagger(train_sentences, backoff=tagger0)
#tagger1.evaluate(test_sentences)#evaluating the accuracy over test dataset
tagger2 = nltk.BigramTagger(train_sentences, backoff=tagger1)
#tagger2.evaluate(test_sentences)#evaluating the accuracy over test dataset
tagger3 = nltk.TrigramTagger(train_sentences, backoff=tagger2)
tagger3.evaluate(test_sentences)#evaluating the accuracy over test dataset


print(fraseTeste)
tags = tagger3.tag(fraseTeste)
print(tags)


In [ ]:
#using NER (Named entity recognition) to identify values in complaints

chunksEntities = nltk.ne_chunk(tags, binary=True)
#namedEntity = nltk.ne_chunk(tags)

#chunksEntities.draw()
#print(chunksEntities)


def returnAllLabels(chunks):
    labels = []
    for chunkEntity in chunks:
        labels.append(chunkEntity[1])
    return labels
    

#function to return the value is been asked for refund or refunded
def getRefundAskedValue(complaint):
    tokensComplaint = Tokenize(complaint)
    tags = tagger3.tag(tokensComplaint)
    chunksEntities = nltk.ne_chunk(tags, binary=True)
    
    labels = returnAllLabels(chunksEntities)
    print(labels)
    #print(len(labels))
    iterator = 0
    while(iterator < len(labels)-1 ) : #the last one label will not be counted.
            #print(labels[iterator] + labels[iterator+1])
            if labels[iterator] == '$' and (labels[iterator+1] == 'NUM'): #search for values preceeded by '$
                entity = chunksEntities[iterator+1]
                entityValue = entity[0]
                print("CHUNKED Monetary Value: " + entityValue)#return the monetary value finded
                return entityValue ##retur
            iterator += 1 
    return 0

def convertStringBrazilCurrencyToFloat(valueString):
    strReplaced = valueString.replace(".","") #removing thousand brazilian separator
    strReplaced = strReplaced.replace(",",".") #replacing brazilian decimal separator for decimal separator
    try:
        return float(strReplaced)
    except:
        pass   
    return 0.0 #return 0 if there is some error in casting

WINDOW_SIZE_CONSTANT =5 #the window size(radio counting from he value) to search for special words
WINDOW_SIZE_CONSTANT_WEIGHT = 3 #the widows size (radio counting from the value) to observe a word that points the value signal 

#function to return the value is been asked for refund
def getRefundAskedValueInComplaint(complaintLematized,setLemmatizedSpecialWords):
    valuesFounded = [0] #list that contains tha asked values founded in the complaints
    
    tokensComplaint = Tokenize(complaintLematized)
    tags = tagger3.tag(tokensComplaint)
    chunksEntities = nltk.ne_chunk(tags, binary=True)
    
    labels = returnAllLabels(chunksEntities)
    #print(len(labels))
    iterator = 0
    while(iterator < len(labels)-1 ) : #the last one label will not be counted.
            #print(labels[iterator] + labels[iterator+1])
            
            if labels[iterator] == '$' and ((labels[iterator+1] == 'NUM') or (labels[iterator+1] == 'N')): #search for values preceeded by '$
                entity = chunksEntities[iterator+1]
                entityValue = entity[0]
                #print("CHUNKED Monetary Value: " + entityValue)#return the monetary value finded
                #if checkSpecialLemasInWindow(tokensComplaint, iterator, setLemmatizedSpecialWords, WINDOW_SIZE_CONSTANT):valuesFounded.append(convertStringBrazilCurrencyToFloat(entityValue))  ##adding the value if there is some special word close it
                
                ##code in case take in count the weight of the value
                if checkSpecialLemasInWindow(tokensComplaint, iterator, setLemmatizedSpecialWords, WINDOW_SIZE_CONSTANT):
                    valueFounded = convertStringBrazilCurrencyToFloat(entityValue)
                    semanticValue = returnSemanticValue(tokensComplaint, iterator, valueFounded, WINDOW_SIZE_CONSTANT_WEIGHT)
                    valuesFounded.append(semanticValue)  #adding the value if there is some special word close it
                    #print("value: "+ str(valueFounded))
                    #print("Semantic value: "+ str(semanticValue))
            
            iterator += 1 
   
    return sum(valuesFounded) #return the sum of the values founded

#function that, based on the position of a monetary value, return if there is a 'special lema' in the window of size n
def checkSpecialLemasInWindow(tokensLemmatized, positionValueToCheck, setSpecialLemas, sizeWindow):
    sizeText = len(tokensLemmatized)
    #firts, lets check for the lemas before the position
    pointer = positionValueToCheck
    while( (pointer != 0) and (positionValueToCheck-pointer <= sizeWindow) ):
        lemaToCheck = tokensLemmatized[pointer]
        if lemaToCheck in setSpecialLemas:
         
            return True
        pointer -= 1 #decrement the value of the pointer
        
    #now, lets check for the lemas after the position
    pointer = positionValueToCheck
    while( (pointer < sizeText) and (pointer-positionValueToCheck <= sizeWindow) ):
        lemaToCheck = tokensLemmatized[pointer]
        if lemaToCheck in setSpecialLemas:
           
            return True
        pointer += 1 #increment the value of the pointer
    
    return False #return false if there is no one special lema in the window

def returnSemanticValue(tokensLemmatized, positionValueToCheck, value, sizeWindow):
    sizeText = len(tokensLemmatized)

    #firts, lets check for the tokens before the position
    pointer = positionValueToCheck
    while( (pointer != 0) and (positionValueToCheck-pointer <= sizeWindow) ):
        lemaToCheck = tokensLemmatized[pointer]
        if lemaToCheck in setLemmasPointSubtraction: ##case subtraction
            return (value * -1)
        pointer -= 1 #decresing the value of the pointer
   
    return value;#return just the value in case not identify subtraction

    #now, lets check for the lemas after the position
    #pointer = positionValueToCheck
    #while( (pointer < sizeText) and (pointer-positionValueToCheck <= sizeWindow) ):
    #    lemaToCheck = tokensLemmatized[pointer]
    #    if lemaToCheck in setSpecialLemas:
     #       #print("Lemma checked After: " + lemaToCheck) 
      #      return True
       # pointer += 1 #increasing the value of the pointer
    
    #return False #return false if there is no one special lema in the window


exampleNoComplaint = 'Quero apenas reclamar dos serviços da operadora, que são muito mais lentos do que eu imaginava.'
exampleComplaint2 = 'Sou titular do telefone (61) 98202-1103. Estou muito chateado com o valor cobrado indevidamente de R$ 450,28 na minha conta. Quero reembolso!'
exampleComplaint3 = "Desejo cancelar minha assinatura, e com isenção da multa de R$ 47,84. "
exampleAnswer1 = "corrigimos a fatura com vencimento 25/05/2020  de R$145,83 para R$118,98"
exampleAnswer2 = "seria isenta a multa rescisória, onde a mesma não será cobrada e realizamos a isenção das faturas de R$69,87 e R$112,18"
exampleAnswer3 = "vamos devoler a diferença  de R$145,83 para R$118,98"

#getRefundAskedValue(preprocess_text(exampleComplaint))
#print(preprocess_text(exampleComplaint2))
#getRefundAskedValueInComplaint(preprocess_text(exampleComplaint2,False), setLemmatizedWordsRefund)
getRefundAskedValueInComplaint(preprocess_text(exampleAnswer3,True), setLemmatizedWordsRefundCompany)
#preprocess_text(exampleAnswer1,True)


In [ ]:
#PREDICTING THE refunded values by the companies
dataFrameRefundAgree = dataFrame.loc[dataFrame['REFUND_AGREE'] == 1]
dataFrameRefundAgree['ESTIMATED_VALUE_REFUNDED'] = dataFrameRefundAgree['TEXT_CLEAN_ANSWERS'].apply(getRefundAskedValue) 

#now let`s analise the accuracy of the refund value prediction just to lines where the prediction was 1
dataFrameTestAccuracyValues = dataFrameRefundAgree.loc[dataFrameRefundAgree['REFUND_AGREE']==1]
dataFrameTestAccuracyValues.head(50)


In [ ]:
dataFrameRefundAgree[['REAL_VALUE_REFUNDED','ESTIMATED_VALUE_REFUNDED']].head(50)

In [ ]:
from nltk import ngrams

sentence = 'The company has R$ 50,00 in debit with me!'

n = 3
sixgrams = ngrams(sentence.split(), n)

for grams in sixgrams:
    print(grams)

In [ ]:
#here, let`s analyze the accuracy of 1200 predictinos using test sample
dataFrameTest = dataFrameTest1200Vectors
testLabels = dataFrameTest.loc[:, ['INCLUDE_REFUND']]
dfTest = dataFrameTest.drop(['INCLUDE_REFUND'], axis=1)

test_x = dfTest
test_y = testLabels 
test_pred = model.predict(test_x)


from sklearn.metrics import accuracy_score, confusion_matrix
print('Accuracy, Confusion Matrix and F1 Score for the AdaBoostClassifier in Test sample of 1200 complaints.  ')
print(accuracy_score(test_y, test_pred))
print(confusion_matrix(test_y,test_pred))
f1 = sklearn.metrics.f1_score(test_y, test_pred)
print('F1 score: %f' % f1)

In [ ]:
#now lets trains the classifer with the vectorized companie answers

from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

train_x, test_x, train_y, test_y = train_test_split(docs_vectors.drop('REFUND_AGREE', axis = 1),
                                                   docs_vectors['REFUND_AGREE'],
                                                   test_size = 0.2,
                                                   random_state = 1)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score

model = AdaBoostClassifier(n_estimators=800, random_state = 1)
model.fit(train_x, train_y)
test_pred = model.predict(test_x)

print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for the AdaBoostClassifier in TRAIN sample of '+str(len(dataFrame)) +' ANSWERS: ')
accuracy = accuracy_score(test_y, test_pred)
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(test_y,test_pred))

f1 = sklearn.metrics.f1_score(test_y,test_pred)
print('F1 score: %f' % f1)
recall = recall_score(test_y,test_pred)
print('Recall score: %f' % recall)
precision = precision_score(test_y,test_pred)
print('Precision score: %f' % precision)

In [ ]:
#now we apply the classification model over the test sample
docs_vectorsTest = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/docsVectors400TestAnswers.csv', sep='\t')
test_x = docs_vectorsTest
test_y = taggedDataFrameTest['REEMBOLSO_CONCEDIDO'].values
test_pred = model.predict(test_x)


print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for the AdaBoostClassifier in TEST sample of '+str(len(test_x)) +' ANSWERS: ')
accuracy = accuracy_score(test_y, test_pred)
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(test_y,test_pred))

f1 = sklearn.metrics.f1_score(test_y,test_pred)
print('F1 score: %f' % f1)
recall = recall_score(test_y,test_pred)
print('Recall score: %f' % recall)
precision = precision_score(test_y,test_pred)
print('Precision score: %f' % precision)

In [ ]:
def convertStringBrazilCurrencyToFloat(valueString):
    strReplaced = valueString.replace(".","") #removing thousand brazilian separator
    strReplaced = strReplaced.replace(",",".") #replacing brazilian decimal separator for decimal separator
    try:
        return float(strReplaced)
    except:
        pass   
    return 0.0

valor = "3.151,70"
convertStringBrazilCurrencyToFloat(valor)

In [ ]:
textExemplePreProcess= 'No dia 18 de abril de 2020, realizei a abertura de um protocolo junto a TIM para verificar o "sumiço" de cerca de R$7,00 reais de crédito que desapareceram de minha linha telefônica (62) 98215-6179. No meu histórico de consumo no site MEUTIM não consta data, hora ou motivo do débito indevido. Em conversa com atendente, ele também não soube me explicar o motivo do desaparecimento do saldo da minha conta e realizou a abertura do protocolo número 2020403961314. Dias depois o protocolo foi dado como finalizado sem a companhia fazer o ressarcimento do valor ou sequer me dar um parecer a respeito do ocorrido. Solicito, a devolução de meus crédito ou, pelo menos, uma explicação sobre o fato.'
preprocess_text(textExemplePreProcess,False)

In [ ]:
#training the classifiers to make the predictiins over CUSTOMER COMPLAINTS

dataFrameTrain = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/docsVectors1200Complaints.csv', sep='\t')

features = dataFrameTrain.columns.difference(['INCLUDE_REFUND'])

#print(features)

#lets point the x as the features to be used, and Y as the targetVariable(INCLUDE_REFUND)
X = dataFrameTrain[features].values
y = dataFrameTrain['INCLUDE_REFUND'].values


#lets train an ADABooster model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

train_x, test_x, train_y, test_y = train_test_split(dataFrameTrain.drop('INCLUDE_REFUND', axis = 1),
                                                   dataFrameTrain['INCLUDE_REFUND'],
                                                   test_size = 0.2,
                                                   random_state = 1)
train_x.shape, train_y.shape, test_x.shape, test_y.shape
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score

model_ADAB = AdaBoostClassifier(n_estimators=800, random_state = 1)
model_ADAB.fit(train_x, train_y)
scores_ADAB = cross_val_score(model_ADAB, X, y, scoring='accuracy', cv=5)
print(scores_ADAB.mean())


#lets train a DECISION TREE model
from sklearn.tree import DecisionTreeClassifier

classifier_dt = DecisionTreeClassifier(random_state=1981, criterion='gini', max_depth=10)
classifier_dt.fit(X, y)#here Decicion Tree model is fitted differently from the others, 

#aplying cross validation to the Decision Tree model
from sklearn.model_selection import cross_val_score
scores_dt = cross_val_score(classifier_dt, X, y, scoring='accuracy', cv=5)
print(scores_dt.mean())


#Constructing a random forest classfier
from sklearn.ensemble import RandomForestClassifier

classifier_rf =    RandomForestClassifier(random_state=1981, criterion='gini', max_depth=10, n_estimators=800, n_jobs=-1)
classifier_rf.fit(train_x, train_y) 

#aplying cross validation to the Ramdom Forest model
scores_rf = cross_val_score(classifier_rf, X, y, scoring='accuracy', cv=5)
print(scores_rf.mean())

#analising the festure importance in the random forest model
#features_importance = zip(classifier_rf.feature_importances_, features)
#for importance, feature in sorted(features_importance, reverse=True):
#    print("%s: %f%%" % (feature, importance*100))
    
#Constructing a GradientBoosting classfier
from sklearn.ensemble import GradientBoostingClassifier

classifier_GB = GradientBoostingClassifier(n_estimators=800, random_state = 1)
classifier_GB.fit(train_x, train_y)
#applying cross validation
scores_GB = cross_val_score(classifier_GB, X, y, scoring='accuracy', cv=5)
print(scores_GB.mean())

In [ ]:
#here let's use the TEST dataset to make the predictions over CUSTOMER COMPLAINTS using different classifiers

docs_vectorsTest = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/docsVectors400TestComplaints.csv', sep='\t')
test_x = docs_vectorsTest
test_y = taggedDataFrameTest['REEMBOLSO_SOLICITADO'].values #test labels

#ADA_BOOSTING
test_pred_ADAB = model_ADAB.predict(test_x)
print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for the AdaBoostClassifier in TEST sample of '+str(len(test_x)) +' complaints: ')
accuracy = accuracy_score(test_y, test_pred_ADAB)
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(test_y,test_pred_ADAB))

f1 = sklearn.metrics.f1_score(test_y,test_pred_ADAB)
print('F1 score: %f' % f1)
recall = recall_score(test_y,test_pred_ADAB)
print('Recall score: %f' % recall)
precision = precision_score(test_y,test_pred_ADAB)
print('Precision score: %f' % precision)
print('\n')


#DECISION TREE
test_pred_DTree = classifier_dt.predict(test_x)
print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for the DecisionTree_Classifier in TEST sample of '+str(len(test_x)) +' COMPLAINTS: ')
accuracy = accuracy_score(test_y, test_pred_DTree)
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(test_y, test_pred_DTree))
f1 = sklearn.metrics.f1_score(test_y, test_pred_DTree)
print('F1 score: %f' % f1)
recall = recall_score(test_y,test_pred_DTree)
print('Recall score: %f' % recall)
precision = precision_score(test_y,test_pred_DTree)
print('Precision score: %f' % precision)

print('\n')
#RANDOM FOREST
test_pred_RandForst = classifier_rf.predict(test_x)
print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for the RandomForest_Classifier in TEST sample of '+str(len(test_x)) +' COMPLAINTS: ')
accuracy = accuracy_score(test_y, test_pred_RandForst)
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(test_y, test_pred_RandForst))
f1 = sklearn.metrics.f1_score(test_y, test_pred_RandForst)
print('F1 score: %f' % f1)
recall = recall_score(test_y,test_pred_RandForst)
print('Recall score: %f' % recall)
precision = precision_score(test_y,test_pred_RandForst)
print('Precision score: %f' % precision)

print('\n')
#GRADIENT BOOSTING
classifier_GB
test_pred_GB = classifier_GB.predict(test_x)
print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for the GradientBoosting_Classifier in TEST sample of '+str(len(test_x)) +' COMPLAINTS: ')
accuracy = accuracy_score(test_y, test_pred_GB)
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(test_y, test_pred_GB))
f1 = sklearn.metrics.f1_score(test_y, test_pred_GB)
print('F1 score: %f' % f1)
recall = recall_score(test_y,test_pred_GB)
print('Recall score: %f' % recall)
precision = precision_score(test_y,test_pred_GB)
print('Precision score: %f' % precision)

In [ ]:
#training the classifiers to make the predictiins over COMPANY ANSWERS

dataFrameTrain = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/docsVectors1200Answers.csv', sep='\t')

features = dataFrameTrain.columns.difference(['REFUND_AGREE'])

from sklearn.model_selection import cross_val_score
#print(features)

#lets point the x as the features to be used, and Y as the targetVariable(REFUND_AGREE)
X = dataFrameTrain[features].values
y = dataFrameTrain['REFUND_AGREE'].values


#lets train an ADABooster model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

train_x, test_x, train_y, test_y = train_test_split(dataFrameTrain.drop('REFUND_AGREE', axis = 1),
                                                   dataFrameTrain['REFUND_AGREE'],
                                                   test_size = 0.2,
                                                   random_state = 1)
train_x.shape, train_y.shape, test_x.shape, test_y.shape
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score

model_ADAB = AdaBoostClassifier(n_estimators=800, random_state = 1)
model_ADAB.fit(train_x, train_y)
scores_ADAB = cross_val_score(model_ADAB, X, y, scoring='accuracy', cv=5)
print(scores_ADAB.mean())


#lets train a DECISION TREE model
from sklearn.tree import DecisionTreeClassifier

classifier_dt = DecisionTreeClassifier(random_state=1981, criterion='gini', max_depth=10)
classifier_dt.fit(train_x, train_y)#here Decicion Tree model is fitted differently from the others, 

#aplying cross validation to the Decision Tree model
from sklearn.model_selection import cross_val_score
scores_dt = cross_val_score(classifier_dt, X, y, scoring='accuracy', cv=5)
print(scores_dt.mean())


#Constructing a random forest classfier
from sklearn.ensemble import RandomForestClassifier

classifier_rf =    RandomForestClassifier(random_state=1981, criterion='gini', max_depth=10, n_estimators=800, n_jobs=-1)
classifier_rf.fit(train_x, train_y) 

#aplying cross validation to the Ramdom Forest model
scores_rf = cross_val_score(classifier_rf, X, y, scoring='accuracy', cv=5)
print(scores_rf.mean())

#analising the festure importance in the random forest model
#features_importance = zip(classifier_rf.feature_importances_, features)
#for importance, feature in sorted(features_importance, reverse=True):
#    print("%s: %f%%" % (feature, importance*100))
    
#Constructing a GradientBoosting classfier
from sklearn.ensemble import GradientBoostingClassifier

classifier_GB = GradientBoostingClassifier(n_estimators=800, random_state = 1)
classifier_GB.fit(train_x, train_y)
#applying cross validation
scores_GB = cross_val_score(classifier_GB, X, y, scoring='accuracy', cv=5)
print(scores_GB.mean())

In [ ]:
#here let's use the TEST dataset to make the predictions over COMPANIES ANSWERS

docs_vectorsTest = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/docsVectors400TestAnswers.csv', sep='\t')
test_x = docs_vectorsTest
test_y = taggedDataFrameTest['REEMBOLSO_CONCEDIDO'].values

#ADA_BOOSTING
test_pred_ADAB = model_ADAB.predict(test_x)
print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for the AdaBoostClassifier in TEST sample of '+str(len(test_x)) +' ANSWERS: ')
accuracy = accuracy_score(test_y, test_pred_ADAB)
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(test_y,test_pred_ADAB))
f1 = sklearn.metrics.f1_score(test_y,test_pred_ADAB)
print('F1 score: %f' % f1)
recall = recall_score(test_y,test_pred_ADAB)
print('Recall score: %f' % recall)
precision = precision_score(test_y,test_pred_ADAB)
print('Precision score: %f' % precision)
print('\n')

#DECISION TREE
test_pred_DTree = classifier_dt.predict(test_x)
print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for the DecisionTree_Classifier in TEST sample of '+str(len(test_x)) +' ANSWERS: ')
accuracy = accuracy_score(test_y, test_pred_DTree)
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(test_y, test_pred_DTree))
f1 = sklearn.metrics.f1_score(test_y, test_pred_DTree)
print('F1 score: %f' % f1)
recall = recall_score(test_y,test_pred_DTree)
print('Recall score: %f' % recall)
precision = precision_score(test_y,test_pred_DTree)
print('Precision score: %f' % precision)
print('\n')


#RANDOM FOREST
test_pred_RandForst = classifier_rf.predict(test_x)
print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for the RandomForest_Classifier in TEST sample of '+str(len(test_x)) +' ANSWERS: ')
accuracy = accuracy_score(test_y, test_pred_RandForst)
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(test_y, test_pred_RandForst))
f1 = sklearn.metrics.f1_score(test_y, test_pred_RandForst)
print('F1 score: %f' % f1)
recall = recall_score(test_y,test_pred_RandForst)
print('Recall score: %f' % recall)
precision = precision_score(test_y,test_pred_RandForst)
print('Precision score: %f' % precision)

print('\n')

#GRADIENT BOOSTING
test_pred_GB = classifier_GB.predict(test_x)
print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for the GradientBoosting_Classifier in TEST sample of '+str(len(test_x)) +' ANSWERS: ')
accuracy = accuracy_score(test_y, test_pred_GB)
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(test_y, test_pred_GB))
f1 = sklearn.metrics.f1_score(test_y, test_pred_GB)
print('F1 score: %f' % f1)
recall = recall_score(test_y,test_pred_GB)
print('Recall score: %f' % recall)
precision = precision_score(test_y,test_pred_GB)
print('Precision score: %f' % precision)

In [ ]:
dataFrameTrain = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/docsVectors1200Answers.csv', sep='\t')
dataFrameTrain.head(2)

In [ ]:
answer = "corrigimos a fatura com vencimento 25/05/2020  de R$145,83 para R$118,98"
#answer = "seria isenta a multa rescisória, onde a mesma não será cobrada e realizamos a isenção das faturas de R$69,87 e R$112,18"
cleanedAnswer = preprocess_text(answer,False)
value = getRefundAskedValueInComplaint(cleanedAnswer, setLemmatizedWordsRefundCompany)
print(answer)
print(cleanedAnswer)

print(value)

In [ ]:

dataFrameTrain = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/docsVectors1200Answers.csv', sep='\t')
     
features = dataFrameTrain.columns.difference(['REFUND_AGREE'])

from sklearn.model_selection import cross_val_score
#print(features)

#lets point the x as the features to be used, and Y as the targetVariable(REFUND_AGREE)
X = dataFrameTrain[features].values
y = dataFrameTrain['REFUND_AGREE'].values


#lets train an ADABooster model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

train_x, test_x, train_y, test_y = train_test_split(dataFrameTrain.drop('REFUND_AGREE', axis = 1),
                                                   dataFrameTrain['REFUND_AGREE'],
                                                   test_size = 0.2,
                                                   random_state = 1)
train_x.shape, train_y.shape, test_x.shape, test_y.shape
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score

model_ADAB = AdaBoostClassifier(n_estimators=800, random_state = 1)
model_ADAB.fit(train_x, train_y)
scores_ADAB = cross_val_score(model_ADAB, X, y, scoring='accuracy', cv=5)
print(scores_ADAB.mean())


#lets train a DECISION TREE model
from sklearn.tree import DecisionTreeClassifier

classifier_dt = DecisionTreeClassifier(random_state=1981, criterion='gini', max_depth=10)
classifier_dt.fit(X, y)

#aplying cross validation to the Decision Tree model
from sklearn.model_selection import cross_val_score
scores_dt = cross_val_score(classifier_dt, X, y, scoring='accuracy', cv=5)
print(scores_dt.mean())


#Constructing a random forest classfier
from sklearn.ensemble import RandomForestClassifier

classifier_rf =    RandomForestClassifier(random_state=1981, criterion='gini', max_depth=10, n_estimators=50, n_jobs=-1)
classifier_rf.fit(X, y) 

#aplying cross validation to the Ramdom Forest model
scores_rf = cross_val_score(classifier_rf, X, y, scoring='accuracy', cv=5)
print(scores_rf.mean())

#analising the festure importance in the random forest model
#features_importance = zip(classifier_rf.feature_importances_, features)
#for importance, feature in sorted(features_importance, reverse=True):
#    print("%s: %f%%" % (feature, importance*100))
    
#Constructing a GradientBoosting classfier
from sklearn.ensemble import GradientBoostingClassifier

classifier_GB = GradientBoostingClassifier(n_estimators=800, random_state = 1)
classifier_GB.fit(X, y)
#applying cross validation
scores_GB = cross_val_score(classifier_GB, X, y, scoring='accuracy', cv=5)
print(scores_GB.mean())

In [ ]:

dataFrameTrain = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/docsVectors1200Answers.csv', sep='\t')

features = dataFrameTrain.columns.difference(['REFUND_AGREE'])

from sklearn.model_selection import cross_val_score
#print(features)

#lets point the x as the features to be used, and Y as the targetVariable(REFUND_AGREE)
X = dataFrameTrain[features].values
y = dataFrameTrain['REFUND_AGREE'].values

train_x, test_x, train_y, test_y = train_test_split(dataFrameTrain.drop('REFUND_AGREE', axis = 1),
                                                   dataFrameTrain['REFUND_AGREE'],
                                                   test_size = 0.2,
                                                   random_state = 1)
train_x.shape, train_y.shape, test_x.shape, test_y.shape
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score

#model_ADAB.fit(train_x, train_y)

#lets train a DECISION TREE model
from sklearn.tree import DecisionTreeClassifier

classifier_dt = DecisionTreeClassifier(random_state=1981, criterion='gini', max_depth=15)
classifier_dt.fit(train_x, train_y)

#aplying cross validation to the Decision Tree model
from sklearn.model_selection import cross_val_score
scores_dt = cross_val_score(classifier_dt, X, y, scoring='accuracy', cv=5)
print(scores_dt.mean())


docs_vectorsTest = pd.read_csv(r'C:/Users/leandro.lira/Desktop/arquivos dissertacao Leandro/docsVectors400TestAnswers.csv', sep='\t')
test_x = docs_vectorsTest
test_y = taggedDataFrameTest['REEMBOLSO_CONCEDIDO'].values

#GRADIENT BOOSTING
test_pred_GB = classifier_dt.predict(test_x)
print('Accuracy, Confusion Matrix, F1 Score, Recall and Precision for the  in TEST sample of '+str(len(test_x)) +' ANSWERS: ')
accuracy = accuracy_score(test_y, test_pred_GB)
print('Accuracy score: %f' % accuracy)
print('Confusion Matrix:')
print(confusion_matrix(test_y, test_pred_GB))
f1 = sklearn.metrics.f1_score(test_y, test_pred_GB)
print('F1 score: %f' % f1)
recall = recall_score(test_y,test_pred_GB)
print('Recall score: %f' % recall)
precision = precision_score(test_y,test_pred_GB)
print('Precision score: %f' % precision)
